In [150]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import polars as pl

In [151]:
path = r"C:\Users\Rudra\Desktop\laptop-price-predictor\data\amazon_laptop_1.csv"

df = pd.read_csv(path)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   product_title           22 non-null     object 
 1   rating                  21 non-null     float64
 2   total_rating            21 non-null     object 
 3   price                   22 non-null     object 
 4   next_month_bought       17 non-null     object 
 5   offer                   21 non-null     object 
 6   mrp                     21 non-null     object 
 7   product_details_dict    22 non-null     object 
 8   brand                   22 non-null     object 
 9   model_name              22 non-null     object 
 10  screen_size             22 non-null     object 
 11  colour                  22 non-null     object 
 12  hard_disk_size          22 non-null     object 
 13  cpu_model               22 non-null     object 
 14  ram                     22 non-null     obje

## MRP

In [152]:
dl = pl.read_csv(path)

In [153]:
dl['mrp'].head(3)

mrp
str
"""₹49,950"""
"""₹1,34,990"""
"""₹12,090"""


In [154]:
dl = dl.with_columns(
    pl.col("mrp")
      .str.replace_all(r'[₹,",]', "")   # remove ₹ , and "
      .cast(pl.Int64)
)

dl['mrp'].head(3)

mrp
i64
49950
134990
12090


## Next month bought

In [155]:
dl = dl.with_columns(
    pl.col("next_month_bought")
      .str.extract(r"(\d+)")
      .cast(pl.Int64)
      .alias("bought_past_month")
)

In [156]:
dl['total_rating']

total_rating
str
"""(120)"""
"""(7)"""
"""(55)"""
"""(7)"""
"""(78)"""
…
"""(202)"""
"""(47)"""
"""(90)"""


In [157]:
# For debugging
dl.select(
    pl.col("total_rating"),
    pl.col("total_rating").str.extract(r"\d+").alias("check")
)

total_rating,check
str,str
"""(120)""",null
"""(7)""",null
"""(55)""",null
"""(7)""",null
"""(78)""",null
…,…
"""(202)""",null
"""(47)""",null
"""(90)""",null


In [158]:
dl = dl.with_columns(
    pl.col("total_rating")
      .cast(pl.Utf8)
      .str.replace_all(r"[^\d]", "")
      .cast(pl.Int16)
      .alias("total_rating")
)

dl['total_rating'].head(3)

total_rating
i16
120
7
55


## Price

In [159]:
dl.head(2)

product_title,rating,total_rating,price,next_month_bought,offer,mrp,product_details_dict,brand,model_name,screen_size,colour,hard_disk_size,cpu_model,ram,os,special_features,graphics_card,about,technical_details_dict,bought_past_month
str,f64,i16,str,str,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,i64
"""Samsung Galaxy Book4 (Gray, 16…",4.1,120,"""45,049.""","""100+ boughtin past month""","""-37%""",49950,"""{'Brand': 'Samsung', 'Model Na…","""Samsung""","""Samsung Galaxy Book4""","""15.6 Inches""","""Gray""","""512 GB""","""Intel Core i5""","""16 GB""","""Windows 11 Home""","""Fingerprint Reader, HD Audio, …","""Integrated""","""About this itemProcessor: Inte…","""{'Brand': '\u200eSamsung', 'Ma…",100
"""ASUS Zenbook S16, AMD Ryzen AI…",3.7,7,"""1,39,990.""",null,"""-21%""",134990,"""{'Brand': 'ASUS', 'Model Name'…","""ASUS""","""ASUS Zenbook S 16""","""16 Inches""","""Scandinavian White""","""1 TB""","""AMD Ryzen 7""","""24 GB""","""Windows 11 Home""","""500nits HDR peak brightness, B…","""Integrated""","""About this itemProcessor : AMD…","""{'Brand': '\u200eASUS', 'Manuf…",null


In [160]:
dl = dl.with_columns(
    pl.col("price")
      .str.replace_all(r'[₹,",.,]', "")   # remove ₹ , and ", .
      .cast(pl.Int64)
)

dl['price'].head(3)

price
i64
45049
139990
12290


## Offer

In [161]:
dl = dl.with_columns(
    pl.col("offer")
      .str.replace_all(r'[₹,",.,%,-]', "")   # remove ₹ , and ", .
      .cast(pl.Int64)
      .alias("offer_pct")
)

dl['offer_pct'].head(3)

offer_pct
i64
37
21
60


In [162]:
dl.with_columns(
    pl.col("brand")
      .str.replace_all(r'[₹,",.,%,-,"]', "")   # remove ₹ , and ", .
)

product_title,rating,total_rating,price,next_month_bought,offer,mrp,product_details_dict,brand,model_name,screen_size,colour,hard_disk_size,cpu_model,ram,os,special_features,graphics_card,about,technical_details_dict,bought_past_month,offer_pct
str,f64,i16,i64,str,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,i64
"""Samsung Galaxy Book4 (Gray, 16…",4.1,120,45049,"""100+ boughtin past month""","""-37%""",49950,"""{'Brand': 'Samsung', 'Model Na…","""Samsung""","""Samsung Galaxy Book4""","""15.6 Inches""","""Gray""","""512 GB""","""Intel Core i5""","""16 GB""","""Windows 11 Home""","""Fingerprint Reader, HD Audio, …","""Integrated""","""About this itemProcessor: Inte…","""{'Brand': '\u200eSamsung', 'Ma…",100,37
"""ASUS Zenbook S16, AMD Ryzen AI…",3.7,7,139990,null,"""-21%""",134990,"""{'Brand': 'ASUS', 'Model Name'…","""ASUS""","""ASUS Zenbook S 16""","""16 Inches""","""Scandinavian White""","""1 TB""","""AMD Ryzen 7""","""24 GB""","""Windows 11 Home""","""500nits HDR peak brightness, B…","""Integrated""","""About this itemProcessor : AMD…","""{'Brand': '\u200eASUS', 'Manuf…",null,21
"""BrowseBook 14.1"" FHD IPS Lapto…",3.1,55,12290,"""100+ boughtin past month""","""-60%""",12090,"""{'Brand': 'Neopticon', 'Model …","""Neopticon""","""BrowseBook""","""14.1 Inches""","""Gray""","""128 GB""","""Celeron N4020""","""4 GB""","""Windows 11 Home""","""HD Audio, Memory Card Slot""","""Integrated""","""About this itemFull HD IPS Dis…","""{'Brand': '\u200eNeopticon', '…",100,60
"""HP 14s Intel Celeron Dual Core…",3.6,7,34999,"""200+ boughtin past month""","""-14%""",25990,"""{'Brand': 'HP', 'Model Name': …","""HP""","""DQ3149TU""","""14 Inches""","""Black""","""512 GB""","""Celeron""","""8 GB""","""Windows 11 Home""","""Anti Glare Coating, Lightweigh…","""Integrated""","""About this itemProcessor - Int…","""{'Brand': '\u200eHP', 'Manufac…",200,14
"""HP OmniBook 5 OLED (Previously…",4.2,78,64990,"""50+ boughtin past month""","""-25%""",65990,"""{'Brand': 'HP', 'Model Name': …","""HP""","""HP Laptop""","""35.6 Centimetres""","""Silver""","""1 TB""","""Snapdragon""","""16 GB""","""Windows 11 Home""","""Anti Glare Coating, Backlit Ke…","""Integrated""","""About this itemProcessor, Memo…","""{'Brand': '\u200eHP', 'Manufac…",50,25
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""ASUS Vivobook Go 14, AMD Ryzen…",4.1,202,31400,"""100+ boughtin past month""","""-30%""",30990,"""{'Brand': 'ASUS', 'Model Name'…","""ASUS""","""ASUS Vivobook Go 14""","""14 Inches""","""Mixed Black""","""512 GB""","""Ryzen 3""","""8 GB""","""Windows 11 Home""","""60Hz refresh rate, Anti-glare …","""Integrated""","""About this itemProcessor : AMD…","""{'Brand': '\u200eASUS', 'Manuf…",100,30
"""ASUS Gaming V16 (2025) 14th Ge…",4.0,47,89990,"""50+ boughtin past month""","""-24%""",86990,"""{'Brand': 'ASUS', 'Model Name'…","""ASUS""","""ASUS Vivobook 16""","""16 Inches""","""Black | Core 7 240H""","""512 GB""","""Intel Core 7""","""16 GB""","""Windows 11 Home""","""Anti-glare display, Backlit Ke…","""Integrated""","""About this itemProcessor : Int…","""{'Brand': '\u200eASUS', 'Manuf…",50,24
"""ASUS Zenbook 14, Intel Core Ul…",4.0,90,85990,null,"""-30%""",94990,"""{'Brand': 'ASUS', 'Model Name'…","""ASUS""","""ASUS Zenbook 14""","""14 Inches""","""Ponder Blue (Ultra 5)""","""1 TB""","""Intel Core Ultra 5""","""16 GB""","""Windows 11 Home""","""120Hz refresh rate, 500nits HD…","""Integrated""","""About this itemProcessor : Int…","""{'Brand': '\u200eASUS', 'Manuf…",null,30


## brand

In [163]:
dl = dl.with_columns(
    pl.col("brand")
    .str.replace_all('"', '')
)
dl.head(3)

product_title,rating,total_rating,price,next_month_bought,offer,mrp,product_details_dict,brand,model_name,screen_size,colour,hard_disk_size,cpu_model,ram,os,special_features,graphics_card,about,technical_details_dict,bought_past_month,offer_pct
str,f64,i16,i64,str,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,i64
"""Samsung Galaxy Book4 (Gray, 16…",4.1,120,45049,"""100+ boughtin past month""","""-37%""",49950,"""{'Brand': 'Samsung', 'Model Na…","""Samsung""","""Samsung Galaxy Book4""","""15.6 Inches""","""Gray""","""512 GB""","""Intel Core i5""","""16 GB""","""Windows 11 Home""","""Fingerprint Reader, HD Audio, …","""Integrated""","""About this itemProcessor: Inte…","""{'Brand': '\u200eSamsung', 'Ma…",100,37
"""ASUS Zenbook S16, AMD Ryzen AI…",3.7,7,139990,null,"""-21%""",134990,"""{'Brand': 'ASUS', 'Model Name'…","""ASUS""","""ASUS Zenbook S 16""","""16 Inches""","""Scandinavian White""","""1 TB""","""AMD Ryzen 7""","""24 GB""","""Windows 11 Home""","""500nits HDR peak brightness, B…","""Integrated""","""About this itemProcessor : AMD…","""{'Brand': '\u200eASUS', 'Manuf…",null,21
"""BrowseBook 14.1"" FHD IPS Lapto…",3.1,55,12290,"""100+ boughtin past month""","""-60%""",12090,"""{'Brand': 'Neopticon', 'Model …","""Neopticon""","""BrowseBook""","""14.1 Inches""","""Gray""","""128 GB""","""Celeron N4020""","""4 GB""","""Windows 11 Home""","""HD Audio, Memory Card Slot""","""Integrated""","""About this itemFull HD IPS Dis…","""{'Brand': '\u200eNeopticon', '…",100,60


In [164]:
dl['brand'].value_counts()

brand,count
str,u32
"""HP""",8
"""Neopticon""",1
"""Lenovo""",3
"""Generic""",1
"""ASUS""",4
"""Samsung""",4
"""acer""",1


## Screen size

In [165]:
dl.select(
    pl.col("screen_size"),
    pl.col("screen_size").str.len_chars().alias("length")
)


screen_size,length
str,u32
"""15.6 Inches""",11
"""16 Inches""",9
"""14.1 Inches""",11
"""14 Inches""",9
"""35.6 Centimetres""",16
…,…
"""14 Inches""",9
"""16 Inches""",9
"""14 Inches""",9


In [166]:
dl['screen_size'].value_counts()

screen_size,count
str,u32
"""39.6 Centimetres""",4
"""35.6 Centimetres""",1
"""16 Inches""",3
"""40.6 Centimetres""",1
"""15.6 Inches""",9
"""14.1 Inches""",1
"""14 Inches""",3


In [167]:
dl = dl.with_columns(
    pl.when(pl.col("screen_size").str.contains("Centimetres"))
      .then(
          pl.col("screen_size")
            .str.split(" ")
            .list.first()
            .cast(pl.Float32)
            * 0.393701
      )
      .otherwise(
          pl.col("screen_size")
            .str.split(" ")
            .list.first()
            .cast(pl.Float32)
      )
      .alias("screen_size_inches")
)

dl.head(2)

product_title,rating,total_rating,price,next_month_bought,offer,mrp,product_details_dict,brand,model_name,screen_size,colour,hard_disk_size,cpu_model,ram,os,special_features,graphics_card,about,technical_details_dict,bought_past_month,offer_pct,screen_size_inches
str,f64,i16,i64,str,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,i64,f32
"""Samsung Galaxy Book4 (Gray, 16…",4.1,120,45049,"""100+ boughtin past month""","""-37%""",49950,"""{'Brand': 'Samsung', 'Model Na…","""Samsung""","""Samsung Galaxy Book4""","""15.6 Inches""","""Gray""","""512 GB""","""Intel Core i5""","""16 GB""","""Windows 11 Home""","""Fingerprint Reader, HD Audio, …","""Integrated""","""About this itemProcessor: Inte…","""{'Brand': '\u200eSamsung', 'Ma…",100,37,15.6
"""ASUS Zenbook S16, AMD Ryzen AI…",3.7,7,139990,null,"""-21%""",134990,"""{'Brand': 'ASUS', 'Model Name'…","""ASUS""","""ASUS Zenbook S 16""","""16 Inches""","""Scandinavian White""","""1 TB""","""AMD Ryzen 7""","""24 GB""","""Windows 11 Home""","""500nits HDR peak brightness, B…","""Integrated""","""About this itemProcessor : AMD…","""{'Brand': '\u200eASUS', 'Manuf…",null,21,16.0


## color

In [168]:
dl['colour'].to_list()

['Gray',
 'Scandinavian White',
 'Gray',
 'Black',
 'Silver',
 'Ci5-16GB Silver',
 'Grey',
 'Blue',
 'Steel Gray',
 'Cloud Grey',
 'Gray',
 'Gray',
 'Natural Silver',
 'Glacier Silver',
 'Natural Silver',
 'Silver',
 'Gray',
 'Mixed Black',
 'Black | Core 7 240H',
 'Ponder Blue (Ultra 5)',
 'Gray',
 'R7 7735HS / Office 2024']

In [169]:
colour_map = {
    "silver|platinum": "Silver",
    "grey|gray|graphite": "Gray",
    "black": "Black",
    "blue": "Blue",
    "white": "White",
}

expr = pl.lit("Other")   # default value

for pattern, value in colour_map.items():
    expr = (
        pl.when(
            pl.col("colour").is_not_null()
            & pl.col("colour").str.to_lowercase().str.contains(pattern)
        )
        .then(pl.lit(value))   
        .otherwise(expr)
    )

dl = dl.with_columns(expr.alias("colour_clean"))


In [173]:
dl.head(3)

product_title,rating,total_rating,price,next_month_bought,offer,mrp,product_details_dict,brand,model_name,screen_size,colour,hard_disk_size,cpu_model,ram,os,special_features,graphics_card,about,technical_details_dict,bought_past_month,offer_pct,screen_size_inches,colour_clean
str,f64,i16,i64,str,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,i64,f32,str
"""Samsung Galaxy Book4 (Gray, 16…",4.1,120,45049,"""100+ boughtin past month""","""-37%""",49950,"""{'Brand': 'Samsung', 'Model Na…","""Samsung""","""Samsung Galaxy Book4""","""15.6 Inches""","""Gray""","""512 GB""","""Intel Core i5""","""16 GB""","""Windows 11 Home""","""Fingerprint Reader, HD Audio, …","""Integrated""","""About this itemProcessor: Inte…","""{'Brand': '\u200eSamsung', 'Ma…",100,37,15.6,"""Gray"""
"""ASUS Zenbook S16, AMD Ryzen AI…",3.7,7,139990,null,"""-21%""",134990,"""{'Brand': 'ASUS', 'Model Name'…","""ASUS""","""ASUS Zenbook S 16""","""16 Inches""","""Scandinavian White""","""1 TB""","""AMD Ryzen 7""","""24 GB""","""Windows 11 Home""","""500nits HDR peak brightness, B…","""Integrated""","""About this itemProcessor : AMD…","""{'Brand': '\u200eASUS', 'Manuf…",null,21,16.0,"""White"""
"""BrowseBook 14.1"" FHD IPS Lapto…",3.1,55,12290,"""100+ boughtin past month""","""-60%""",12090,"""{'Brand': 'Neopticon', 'Model …","""Neopticon""","""BrowseBook""","""14.1 Inches""","""Gray""","""128 GB""","""Celeron N4020""","""4 GB""","""Windows 11 Home""","""HD Audio, Memory Card Slot""","""Integrated""","""About this itemFull HD IPS Dis…","""{'Brand': '\u200eNeopticon', '…",100,60,14.1,"""Gray"""


## Hard disk

In [176]:
dl['hard_disk_size'].value_counts()

hard_disk_size,count
str,u32
"""128 GB""",1
"""512 GB""",16
"""256 GB""",1
"""1 TB""",4


In [181]:
dl = dl.with_columns(
    pl.when(pl.col("hard_disk_size").str.contains("TB"))
        .then(
            pl.col("hard_disk_size")
            .str.split(" ")
            .list.first()
            .cast(pl.Int16)
            * 1000
            )
        .otherwise(
            pl.col("hard_disk_size")
            .str.split(" ")
            .list.first()
            .cast(pl.Int16)
            
            )
        .alias("hdd_in_gb")
    )

In [182]:
dl.head(2)

product_title,rating,total_rating,price,next_month_bought,offer,mrp,product_details_dict,brand,model_name,screen_size,colour,hard_disk_size,cpu_model,ram,os,special_features,graphics_card,about,technical_details_dict,bought_past_month,offer_pct,screen_size_inches,colour_clean,hdd_in_gb
str,f64,i16,i64,str,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,i64,f32,str,i16
"""Samsung Galaxy Book4 (Gray, 16…",4.1,120,45049,"""100+ boughtin past month""","""-37%""",49950,"""{'Brand': 'Samsung', 'Model Na…","""Samsung""","""Samsung Galaxy Book4""","""15.6 Inches""","""Gray""","""512 GB""","""Intel Core i5""","""16 GB""","""Windows 11 Home""","""Fingerprint Reader, HD Audio, …","""Integrated""","""About this itemProcessor: Inte…","""{'Brand': '\u200eSamsung', 'Ma…",100,37,15.6,"""Gray""",512
"""ASUS Zenbook S16, AMD Ryzen AI…",3.7,7,139990,null,"""-21%""",134990,"""{'Brand': 'ASUS', 'Model Name'…","""ASUS""","""ASUS Zenbook S 16""","""16 Inches""","""Scandinavian White""","""1 TB""","""AMD Ryzen 7""","""24 GB""","""Windows 11 Home""","""500nits HDR peak brightness, B…","""Integrated""","""About this itemProcessor : AMD…","""{'Brand': '\u200eASUS', 'Manuf…",null,21,16.0,"""White""",1000


## ram

In [184]:
dl['ram'].value_counts()

ram,count
str,u32
"""12 GB""",1
"""24 GB""",1
"""8 GB""",5
"""4 GB""",1
"""16 GB""",14


In [191]:
dl = dl.with_columns(
    pl.col('ram')
    .str.split(" ")
    .list.first()
    .cast(pl.Int16)
    .alias(r"ram_in_gb")
)

dl.head(3)

product_title,rating,total_rating,price,next_month_bought,offer,mrp,product_details_dict,brand,model_name,screen_size,colour,hard_disk_size,cpu_model,ram,os,special_features,graphics_card,about,technical_details_dict,bought_past_month,offer_pct,screen_size_inches,colour_clean,hdd_in_gb,ram_in_gb
str,f64,i16,i64,str,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,i64,f32,str,i16,i16
"""Samsung Galaxy Book4 (Gray, 16…",4.1,120,45049,"""100+ boughtin past month""","""-37%""",49950,"""{'Brand': 'Samsung', 'Model Na…","""Samsung""","""Samsung Galaxy Book4""","""15.6 Inches""","""Gray""","""512 GB""","""Intel Core i5""","""16 GB""","""Windows 11 Home""","""Fingerprint Reader, HD Audio, …","""Integrated""","""About this itemProcessor: Inte…","""{'Brand': '\u200eSamsung', 'Ma…",100,37,15.6,"""Gray""",512,16
"""ASUS Zenbook S16, AMD Ryzen AI…",3.7,7,139990,null,"""-21%""",134990,"""{'Brand': 'ASUS', 'Model Name'…","""ASUS""","""ASUS Zenbook S 16""","""16 Inches""","""Scandinavian White""","""1 TB""","""AMD Ryzen 7""","""24 GB""","""Windows 11 Home""","""500nits HDR peak brightness, B…","""Integrated""","""About this itemProcessor : AMD…","""{'Brand': '\u200eASUS', 'Manuf…",null,21,16.0,"""White""",1000,24
"""BrowseBook 14.1"" FHD IPS Lapto…",3.1,55,12290,"""100+ boughtin past month""","""-60%""",12090,"""{'Brand': 'Neopticon', 'Model …","""Neopticon""","""BrowseBook""","""14.1 Inches""","""Gray""","""128 GB""","""Celeron N4020""","""4 GB""","""Windows 11 Home""","""HD Audio, Memory Card Slot""","""Integrated""","""About this itemFull HD IPS Dis…","""{'Brand': '\u200eNeopticon', '…",100,60,14.1,"""Gray""",128,4


## os

In [193]:
dl['os'].value_counts()

os,count
str,u32
"""Windows""",1
"""Windows 11""",1
"""Windows 11 Home""",19
"""DOS""",1


In [ ]:
os_map = {
    "Mac|mac": "Mac",
    "window": "Window",
}

expr = pl.lit("Other")   # default value

for pattern, value in os_map.items():
    expr = (
        pl.when(
            pl.col("os").is_not_null()
            & pl.col("os").str.to_lowercase().str.contains(pattern)
        )
        .then(pl.lit(value))   
        .otherwise(expr)
    )

dl = dl.with_columns(expr.alias("os_name"))
dl.head(1)

product_title,rating,total_rating,price,next_month_bought,offer,mrp,product_details_dict,brand,model_name,screen_size,colour,hard_disk_size,cpu_model,ram,os,special_features,graphics_card,about,technical_details_dict,bought_past_month,offer_pct,screen_size_inches,colour_clean,hdd_in_gb,ram_in_gb,os_name
str,f64,i16,i64,str,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,i64,f32,str,i16,i16,str
"""Samsung Galaxy Book4 (Gray, 16…",4.1,120,45049,"""100+ boughtin past month""","""-37%""",49950,"""{'Brand': 'Samsung', 'Model Na…","""Samsung""","""Samsung Galaxy Book4""","""15.6 Inches""","""Gray""","""512 GB""","""Intel Core i5""","""16 GB""","""Windows 11 Home""","""Fingerprint Reader, HD Audio, …","""Integrated""","""About this itemProcessor: Inte…","""{'Brand': '\u200eSamsung', 'Ma…",100,37,15.6,"""Gray""",512,16,"""Window"""
"""ASUS Zenbook S16, AMD Ryzen AI…",3.7,7,139990,null,"""-21%""",134990,"""{'Brand': 'ASUS', 'Model Name'…","""ASUS""","""ASUS Zenbook S 16""","""16 Inches""","""Scandinavian White""","""1 TB""","""AMD Ryzen 7""","""24 GB""","""Windows 11 Home""","""500nits HDR peak brightness, B…","""Integrated""","""About this itemProcessor : AMD…","""{'Brand': '\u200eASUS', 'Manuf…",null,21,16.0,"""White""",1000,24,"""Window"""
"""BrowseBook 14.1"" FHD IPS Lapto…",3.1,55,12290,"""100+ boughtin past month""","""-60%""",12090,"""{'Brand': 'Neopticon', 'Model …","""Neopticon""","""BrowseBook""","""14.1 Inches""","""Gray""","""128 GB""","""Celeron N4020""","""4 GB""","""Windows 11 Home""","""HD Audio, Memory Card Slot""","""Integrated""","""About this itemFull HD IPS Dis…","""{'Brand': '\u200eNeopticon', '…",100,60,14.1,"""Gray""",128,4,"""Window"""
"""HP 14s Intel Celeron Dual Core…",3.6,7,34999,"""200+ boughtin past month""","""-14%""",25990,"""{'Brand': 'HP', 'Model Name': …","""HP""","""DQ3149TU""","""14 Inches""","""Black""","""512 GB""","""Celeron""","""8 GB""","""Windows 11 Home""","""Anti Glare Coating, Lightweigh…","""Integrated""","""About this itemProcessor - Int…","""{'Brand': '\u200eHP', 'Manufac…",200,14,14.0,"""Black""",512,8,"""Window"""
"""HP OmniBook 5 OLED (Previously…",4.2,78,64990,"""50+ boughtin past month""","""-25%""",65990,"""{'Brand': 'HP', 'Model Name': …","""HP""","""HP Laptop""","""35.6 Centimetres""","""Silver""","""1 TB""","""Snapdragon""","""16 GB""","""Windows 11 Home""","""Anti Glare Coating, Backlit Ke…","""Integrated""","""About this itemProcessor, Memo…","""{'Brand': '\u200eHP', 'Manufac…",50,25,14.015755,"""Silver""",1000,16,"""Window"""


## CPU

In [206]:
dl['cpu_model'].to_list()

['Intel Core i5',
 'AMD Ryzen 7',
 'Celeron N4020',
 'Celeron',
 'Snapdragon',
 'Core i5 Family',
 'Ryzen 3',
 'Ryzen 5',
 'Others',
 'Celeron',
 'Intel Core 5',
 'Ryzen 5',
 'Core i3',
 'Snapdragon X',
 'Core i3 Family',
 'Intel Core i7',
 'Intel Core i5',
 'Ryzen 3',
 'Intel Core 7',
 'Intel Core Ultra 5',
 'Intel Core 7',
 'Ryzen 7']

In [207]:
cpu_map = {
    "Intel|core|Core": "Intel",
    "AMD|ryzen|Ryzen": "AMD",
    "Celeron": "Celeron",
    "Snapdragon": "Snapdragon",
    "MediaTek": "MediaTek",
    "Mac|Apple": "Mac"
}

expr = pl.lit("Other")   # default value

for pattern, value in cpu_map.items():
    expr = (
        pl.when(
            pl.col("cpu_model").is_not_null()
            & pl.col("cpu_model").str.to_lowercase().str.contains(pattern)
        )
        .then(pl.lit(value))   
        .otherwise(expr)
    )

dl = dl.with_columns(expr.alias("cpu_brand"))
dl.head(1)

product_title,rating,total_rating,price,next_month_bought,offer,mrp,product_details_dict,brand,model_name,screen_size,colour,hard_disk_size,cpu_model,ram,os,special_features,graphics_card,about,technical_details_dict,bought_past_month,offer_pct,screen_size_inches,colour_clean,hdd_in_gb,ram_in_gb,os_name,cpu_brand
str,f64,i16,i64,str,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,i64,f32,str,i16,i16,str,str
"""Samsung Galaxy Book4 (Gray, 16…",4.1,120,45049,"""100+ boughtin past month""","""-37%""",49950,"""{'Brand': 'Samsung', 'Model Na…","""Samsung""","""Samsung Galaxy Book4""","""15.6 Inches""","""Gray""","""512 GB""","""Intel Core i5""","""16 GB""","""Windows 11 Home""","""Fingerprint Reader, HD Audio, …","""Integrated""","""About this itemProcessor: Inte…","""{'Brand': '\u200eSamsung', 'Ma…",100,37,15.6,"""Gray""",512,16,"""Window""","""Intel"""


## Graphic card

In [210]:
dl['graphics_card'].to_list()

['Integrated',
 'Integrated',
 'Integrated',
 'Integrated',
 'Integrated',
 'Integrated',
 'Integrated',
 'Dedicated',
 'Integrated',
 'Integrated',
 'Integrated',
 'Integrated',
 'Integrated',
 'Integrated',
 'Integrated',
 'Integrated',
 'Integrated',
 'Integrated',
 'Integrated',
 'Integrated',
 'Integrated',
 'Integrated']

In [208]:
dl[
 'special_features',
 'graphics_card',
 'about',]

special_features,graphics_card,about
str,str,str
"""Fingerprint Reader, HD Audio, …","""Integrated""","""About this itemProcessor: Inte…"
"""500nits HDR peak brightness, B…","""Integrated""","""About this itemProcessor : AMD…"
"""HD Audio, Memory Card Slot""","""Integrated""","""About this itemFull HD IPS Dis…"
"""Anti Glare Coating, Lightweigh…","""Integrated""","""About this itemProcessor - Int…"
"""Anti Glare Coating, Backlit Ke…","""Integrated""","""About this itemProcessor, Memo…"
…,…,…
"""60Hz refresh rate, Anti-glare …","""Integrated""","""About this itemProcessor : AMD…"
"""Anti-glare display, Backlit Ke…","""Integrated""","""About this itemProcessor : Int…"
"""120Hz refresh rate, 500nits HD…","""Integrated""","""About this itemProcessor : Int…"


In [172]:
dl.columns

['product_title',
 'rating',
 'total_rating',
 'price',
 'next_month_bought',
 'offer',
 'mrp',
 'product_details_dict',
 'brand',
 'model_name',
 'screen_size',
 'colour',
 'hard_disk_size',
 'cpu_model',
 'ram',
 'os',
 'special_features',
 'graphics_card',
 'about',
 'technical_details_dict',
 'bought_past_month',
 'offer_pct',
 'screen_size_inches',
 'colour_clean']